# Hispanic image 생성 StyleGAN2-ADA 


StyleGAN2-ADA only work with Tensorflow 1. Run the next cell before anything 

else to make sure we’re using TF1 and not TF2.

In [ ]:
%tensorflow_version 1.x

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Nov  7 06:37:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Install Repo to Google Drive

Colab is a little funky with training. I’ve found the best way to do this is to install the repo directly into your Google Drive folder.

First, mount your Drive to the Colab notebook: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Next, run this cell. If you’re already installed the repo, it will skip the installation process and change into the repo’s directory. If you haven’t installed it, it will install all the files necessary.

In [ ]:
import os
if os.path.isdir("./AIB_04_Stylebot/colab-sg2-ada"):
    %cd "./AIB_04_Stylebot/colab-sg2-ada/stylegan2-ada"
else:
    #install script
    %cd "/content/drive/MyDrive/AIB_04_Stylebot"
    !mkdir colab-sg2-ada
    %cd colab-sg2-ada
    !git clone https://github.com/dvschultz/stylegan2-ada
    %cd stylegan2-ada
    !mkdir downloads
    !mkdir datasets

/content/drive/.shortcut-targets-by-id/1zQGyqDeW63FEr3sXvFz23SVnqvoGEBUd/AIB_04_Stylebot/colab-sg2-ada/stylegan2-ada


In [ ]:
import cv2
from os import listdir
from os.path import isfile, join
from google.colab import files
from PIL import Image, ImageDraw
import numpy as np
from tqdm import tqdm
import imageio
import os
from pathlib import Path
import glob
import matplotlib.pyplot as plt

In [ ]:
# 1024 *1024 resize

result_path = './AIB_04_Stylebot/selected_crop_photo'

image_files = Path(result_path)
image_files = list(image_files.glob(r'*.jpg'))

for name in image_files[:]:
  img = cv2.imread(f'{name}')
  img = cv2.resize(img, (1024,1024))

  # 전처리 이미지 저장
  print(img.shape)

## Convert dataset to .tfrecords

수집한 이미지 데이터를 .tfrecords 파일로 변환해 주는 코드입니다.
* 데이터셋에서 한번만 작동하면 됩니다. 
* tfrecords
=TFRecord는 데이터 세트의 포맷의 하나로 TFRecord형식은 바이너리 코드의 시리즈를 저장하기 위한 단순한 형식 ==> 대규모 데이터를 효율적으로 학습할 수 있게 해줍니다..

In [ ]:
#you don't need to edit anything here
!python dataset_tool.py create_from_images ./stylegan2-ada/datasets/{'1024image'} {result_path}

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
Loading images from "/content/drive/MyDrive/AIB_04_Stylebot/selected_crop"
Creating dataset "/content/drive/MyDrive/AIB_04_Stylebot/colab-sg2-ada/stylegan2-ada/datasets/1024image"
dataset_tool.py:97: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[quant.tostring()]))}))
Added 55 images.


## Train a custom model
* 모델 학습
* pretrained dataset = ffhq1024를 사용 했습니다.

In [ ]:
# dataset_name만 바꾸고 진행했습니다.

#this name must EXACTLY match the dataset name you used when creating the .tfrecords file
dataset_name = "new_image"
#how often should the model generate samples and a .pkl file
snapshot_count = 4
#should the images be mirrored left to right?
mirrored = True
#should the images be mirrored top to bottom?
mirroredY = False
#metrics? 
metric_list = None
#augments
augs = "bgc"

#
# this is the most important cell to update
#
# running it for the first time? set it to ffhq(+resolution)
# resuming? get the path to your latest .pkl file and use that
resume_from = "ffhq1024"

# model training
!python train.py --outdir ./results --snap={snapshot_count} --cfg=stylegan2 --kimg=100 --data=./datasets/{dataset_name} --augpipe={augs} --mirror={mirrored} --mirrory={mirroredY} --metrics={metric_list} --resume={resume_from}

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
tcmalloc: large alloc 4294967296 bytes == 0x558350a92000 @  0x7f97f48ce001 0x7f97f1ad154f 0x7f97f1b21b58 0x7f97f1b25b17 0x7f97f1bc4203 0x558349a22544 0x558349a22240 0x558349a96627 0x558349a90ced 0x558349a2448c 0x558349a65159 0x558349a620a4 0x558349a22d49 0x558349a9694f 0x558349a909ee 0x558349962e2b 0x558349a92fe4 0x558349a909ee 0x558349962e2b 0x558349a92fe4 0x558349a90ced 0x558349962e2b 0x558349a92fe4 0x558349a23afa 0x558349a91915 0x558349a909ee 0x558349a906f3 0x558349b5a4c2 0x558349b5a83d 0x558349b5a6e6 0x558349b32163
tcmalloc: large alloc 4294967296 bytes == 0x558450a92000 @  0x7f97f48cc1e7 0x7f97f1ad146e 0x7f97f1b21c7b 0x7f97f1b2235f 0x7f97f1bc4103 0x558349a22544 0x558349a22240 0x558349a96627 0x558349a909ee 0x558349a23bda 0x558349a92737 0x558349a909ee 0x558349a23bda 0x558349a92737 0x558349a909ee

## Generator


In [ ]:
# 1024해상도의 200kimg 학습된 이미지의 pkl파일
network = './network-snapshot-000200.pkl'

#생성된 이미지 seed 확인인--create-grid
# !python generate.py generate-images --trunc=0.5 --seeds=0-45 --create-grid  --network={network} 

In [ ]:
# interpolation
!python generate.py generate-latent-walk --trunc=0.5 --seeds=0-45 --create-grid  --network={network} 

## Style Mixing



In [ ]:
!python style_mixing.py --trunc=0.5 --rows=14,1000,1022,1080,1086,42994967295 --cols=23,1004,1043,30,1048  --network='./network-snapshot-000200.pkl' --styles=0-3

Loading networks from "/content/drive/MyDrive/AIB_04_Stylebot/network-snapshot-000100.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Generating W vectors...
Generating images...
Generating style-mixed images...
Saving images...
Saving image grid...
